## Sensitivity Analysis Experiment
Try out different values of $\lambda$ and $\alpha$ in the cost function on the 10 images in the dataset. See which seems to perform the best on average. 
1. 10 images * 20 $\alpha$s * 20 $\lambda$s = 4000 optimizations. The algorithm doesn't seem very sensitive to starting point, so we ignore this. 
2. 8 columns, $\lambda, \alpha$, then 5 for the confusion matrices, one for f score 

In [42]:
include("objective3.jl")
include("numerical_statistics.jl")
using Images, FileIO, Statistics, LinearAlgebra

In [43]:
nλ = 3#20
nα = 3#20
n_images = 3#10
λ_values = LinRange(.1, 1, nλ)
α_values = LinRange(.005, .05, nα)
summary_h = zeros(nλ*nα*n_images, 1+1+1 + 5);
summary_v = zeros(nλ*nα*n_images, 1+1+1 + 5);
counter = 1;

In [44]:
for i in 1:n_images
    im_path = "./images2/im"*string(i)*".png"
    hmask_path = "./images2/im"*string(i)*"_hmask.png"
    vmask_path = "./images2/im"*string(i)*"_vmask.png"
    
    # load images
    img = load(im_path);
    imgg = Gray.(img);
    im = convert(Array{Float64}, imgg);

    img_hmask = load(hmask_path);
    imgg_hmask = Gray.(img_hmask);
    im_hmask = convert(Array{Float64}, imgg_hmask);

    img_vmask = load(vmask_path);
    imgg_vmask = Gray.(img_vmask);
    im_vmask = convert(Array{Float64}, imgg_vmask);
    
    for λw in λ_values
        for αw in α_values
            # Set start point of optimization
            imheight, imwidth = size(im) 
            f = .5 .+ randn(imheight, imwidth)/10;
            h = .5 .+ randn(imheight-1, imwidth)/10;
            v = .5 .+ randn(imheight, imwidth-1)/10;

            # ### Interior Point
            niters = 7
            ρs = [1.5625*.2^k for k in 2:niters]
            αws = repeat([αw], niters-1) # .005 seems to work with p, .002 seems to work well with q. .05 maximum
            λs = repeat([λw], niters - 1)
            f,h,v = interior_point(gg, ∇gg, q, ∇q, f, h, v, λs, ρs, αws, im)

            confusion = get_confusion(h,v, im_hmask, im_vmask)
            f_for_both = get_f_score(confusion)
            summary_h[counter, :] .= append!([λw, αw], confusion[1, :], [f_for_both])
            summary_v[counter, :] .= append!([λw, αw], confusion[2, :], [f_for_both])
            print(counter, " done\n")
            counter += 1
        end
    end
end

1 done
2 done
3 done
4 done
5 done
6 done
7 done
8 done
9 done
10 done
11 done
12 done
13 done
14 done
15 done
16 done
17 done
18 done
19 done
20 done
21 done
22 done
23 done
24 done
25 done
26 done
27 done


In [45]:
summary_h

27×8 Matrix{Float64}:
 0.1   0.005   840.0   0.0  0.0  30.0  870.0  0.0
 0.1   0.0275  840.0   0.0  0.0  30.0  870.0  0.0
 0.1   0.05    840.0   0.0  0.0  30.0  870.0  0.0
 0.55  0.005   840.0  21.0  0.0   9.0  870.0  0.885496
 0.55  0.0275  840.0  12.0  0.0  18.0  870.0  0.684685
 0.55  0.05    840.0  11.0  0.0  19.0  870.0  0.635514
 1.0   0.005   840.0  28.0  0.0   2.0  870.0  0.979021
 1.0   0.0275  840.0  12.0  0.0  18.0  870.0  0.684685
 1.0   0.05    840.0  12.0  0.0  18.0  870.0  0.684685
 0.1   0.005   822.0   0.0  0.0  48.0  870.0  0.0
 0.1   0.0275  822.0   0.0  0.0  48.0  870.0  0.0
 0.1   0.05    822.0   0.0  0.0  48.0  870.0  0.0
 0.55  0.005   822.0  31.0  0.0  17.0  870.0  0.836879
 ⋮                                ⋮           
 1.0   0.005   822.0  39.0  0.0   9.0  870.0  0.921053
 1.0   0.0275  822.0  13.0  0.0  35.0  870.0  0.621849
 1.0   0.05    822.0  11.0  0.0  37.0  870.0  0.586207
 0.1   0.005   835.0   0.0  0.0  35.0  870.0  0.0
 0.1   0.0275  835.0   0.0  0.0